
# Generate transcript from Youtube video 
- Summarize Youtube's script by chapter creater configured. 
  - Create `markdown_note.md` with script and summary.
- Use [yt-dlp](https://pypi.org/project/yt-dlp/), [pydub](https://pypi.org/project/pydub/), [OpenAI-Whisper](https://pypi.org/project/openai-whisper/), [langchain](https://github.com/hwchase17/langchain), and [OpenAI](https://github.com/openai/openai-python) package. 


## Input variables

In [ ]:
# Youtube video ID
youtube_video_id="O8GLjpnW-cM"

# Language of subscription 
language = "ko"


# Officially no way to get chapter automatically, 
# so copy and paste the time stamp and chapter in description of Youtube video.
# CAUTION: Timestamp 00:00 must be defined! 
chapter_part_in_description = """
00:00 시작
05:56 전해철의 대의원제에 대한 견해: 억압과 착취의 메커니즘
19:57 문재인 정부에서 꿀 빨던 인간들은... 도덕성을 말하지 마라
34:12 철학, 실력, 용기
48:15 우리는 정말 대전환을 원하는가? 관존민비 → 민존관비: ‘그놈정신’이 필요한 이유
57:21 이재명의 청계광장 연설(2016.10.29.)
1:05:39 노무현의 대통령경선 출마 연설 [새천년민주당, 서울 힐튼호텔(2001.12.10.)]
1:15:31 억압과 착취의 메커니즘: 구조, 시스템, 프로세스, 사람
1:21:34 국가경영의 지배구조(National Governance)
1:31:08 부탁의 말씀, 《더 많은 권력을 시민에게》(토마스 베네딕토, 성연숙 옮김)
1:36:24 노무현의 철학, 실력, 용기, 그리고 그의 사고력
1:39:31 앵글로색슨 모형(억압과 착취의 구조) vs. 게르만 모형(대화와 토론의 구조)
1:42:11 대전환의 의미: ‘그놈정신’으로 무장하라
1:44:00 무엇이 가장 중요한가?
1:44:45 정리
"""

hint_to_fix = \
"""
강진국 -> 강진구
박대원 -> 박대용
도독성 -> 도덕성
윤류의식 -> 윤리의식 
알궏리를 -> 알권리를
어갑하고 착취하도록 -> 억압하고 착취하도록
일어쿽 저러고 -> 이러쿵 저러쿵
홍의 -> 혼외
나라구 -> 나락
도 탐사 -> 더탐사
개몽 -> 계몽
성충죠 -> 성취죠
석진 위원장 -> 혁신 위원장
성취혜층 모형 -> 성취예측모형
강준규 -> 강진구
홍의 자식 -> 혼외자식
고질범 51이라는 -> 고작 51일이라는
총화대에서 -> 청와대에서
문소 -> 문서
사회 통염 -> 사회통념
포난라서 -> 퍼날라서
학교 좀 -> 합격점
조직설리 -> 조직설계
있어 빌리티만 -> 있어빌리티만
문제인 -> 문재인
내 마음에 벗과 싸워 -> 내 마음에 법과 싸워
이제 명예게 -> 이제명에게
"""

In [ ]:
# Officially no way to get chapter automatically, 
# so we need to parse the text in description and set up the dictionary 
# [ (time_in_sec, chapter_title) ]
import re 
pattern = r'(\d+(:\d+){1,2})\s(.+)'
matches = re.findall(pattern, chapter_part_in_description)

def time_to_seconds(time):
    parts = time.split(':')
    seconds = int(parts[-1])
    minutes = int(parts[-2]) if len(parts) > 1 else 0
    hours = int(parts[-3]) if len(parts) > 2 else 0
    return hours * 3600 + minutes * 60 + seconds

chapters = [(time_to_seconds(time), title.strip()) for time, _, title in matches]


# Build up note with chapter and script under each chapter 

In [ ]:
import os
import yt_dlp

# Download youtube video and extract audio file. 
def download(video_id: str) -> str:
    video_url = f'https://www.youtube.com/watch?v={video_id}'
    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'paths': {'home': 'audio/'},
        'outtmpl': {'default': '%(id)s.%(ext)s'},
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }]
    }

    if os.path.exists(f'audio/{video_id}.mp3'):
        return ""

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([video_url])
        if error_code != 0:
            raise Exception('Failed to download video')

    return f'audio/{video_id}.mp3'

file_path = download(youtube_video_id)



In [ ]:
# split audio file
import os
import time
from pydub import AudioSegment

if os.path.exists(file_path) or file_path != '':
    audio_data = AudioSegment.from_mp3(file_path)
    
    for i in range( len(chapters) ):

        current_time_in_sec, current_title = chapters[i]
        next_time_in_sec, next_title = chapters[i + 1] if i + 1 < len(chapters) else (None, None)

        current_time_in_ms = current_time_in_sec * 1000
        next_time_in_ms = next_time_in_sec * 1000 if next_time_in_sec is not None else 0

        if next_time_in_sec:
            splitted_audio_data = audio_data[current_time_in_ms:next_time_in_ms]
        else:
            splitted_audio_data = audio_data[current_time_in_ms:]

        splitted_audio_data.export(f'audio/{i}.mp3' , format="mp3")

In [ ]:
# Transcribe the text from audio files.
import os
from datetime import datetime
from dotenv import load_dotenv

import whisper
import llm

# Setup OpenAI API key 
load_dotenv()

# Prepare the file path for text
text_file_folder_path = os.path.join( os.getcwd(), 'text')
if not os.path.exists( text_file_folder_path ):
    os.makedirs(text_file_folder_path) 


# You can adjust the model used here. Model choice is typically a tradeoff between accuracy and speed.
# All available models are located at https://github.com/openai/whisper/#available-models-and-languages.
whisper_model = whisper.load_model("small")

script_by_chapter = []
def transcribe(file_path: str) -> str:
    # `fp16` defaults to `True`, which tells the model to attempt to run on GPU.
    # For local demonstration purposes, we'll run this on the CPU by setting it to `False`.
    transcription = whisper_model.transcribe(file_path, fp16=False)
    return transcription['text'] # type: ignore

for i in range( len(chapters) ):
    current_time_in_sec, current_title = chapters[i]
    print( f'{datetime.now()} : {current_title} is transcripting... \n' )
    audio_file_path = os.path.join( os.getcwd(), 'audio', f'{i}.mp3' )

    text_file = os.path.join(text_file_folder_path, f'{i}.txt')

    if not os.path.exists(text_file):
        transcript = transcribe(audio_file_path)
        cleaned_transcript = llm.clean_up_sentence_punctuation_and_fix_errors(transcript, 
                                                                            hint_to_fix, 
                                                                            chunk_size=400, 
                                                                            verbose=True)

        merged_text = \
    f"""
    {transcript}

    --------

    {cleaned_transcript}
    """

        chapter_data = { 
                    "title": current_title,
                    "script": merged_text,
                    "summary" : ""
                    }
        
        script_by_chapter.append(chapter_data)

        # Save transcript file

        with open( text_file, "w") as file:
            file.write(chapter_data["script"])



In [ ]:

# Temporary save data into file 
import os 
import json 

with open( "temp_script_by_chapter.json", "w") as file:
    file.write( json.dumps(script_by_chapter, indent=2, ensure_ascii=False) )
